## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [22]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [23]:
import pandas as pd
pd.set_option('display.max_row',300)

### 几种不同的agent类型
### Several different types of agent
- ZERO_SHOT_REACT_DESCRIPTION
- CHAT_ZERO_SHOT_REACT_DESCRIPTION
- CONVERSATIONAL_REACT_DESCRIPTION
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION
- OPENAI_FUNCTIONS
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
<hr>

- ZERO_SHOT_REACT_DESCRIPTION 零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。https://blog.csdn.net/zcyzcyjava/article/details/127006287 [零样本、单样本、少样本]
- Zero-shot generative dialogue, which can autonomously conduct conversations without examples. https://blog.csdn.net/zcyzcyjava/article/details/127006287 [Zero-shot, one-shot, few-shot]

In [24]:
from langchain_openai import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)
#agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")


verbose=True tags=['zero-shot-react-description'] agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'), llm=OpenAI(client=<openai.resources.comp

- CHAT_ZERO_SHOT_REACT_DESCRIPTION 使用了chatmodel
- CHAT_ZERO_SHOT_REACT_DESCRIPTION uses chatmodel

In [25]:
from langchain_openai import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
#agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")


verbose=True tags=['chat-zero-shot-react-description'] agent=ChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Search, Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_N

- CONVERSATIONAL_REACT_DESCRIPTION 一个对话型的agent，这个agent要求与memory一起使用
- CONVERSATIONAL_REACT_DESCRIPTION A conversational agent, this agent requires to be used with memory

In [26]:
from langchain_openai import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
#memory component
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)

memory=ConversationBufferMemory(memory_key='chat_history') verbose=True tags=['conversational-react-description'] agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Ass

In [27]:
agent.invoke("hi i am tomie")



> Entering new AgentExecutor chain...
Thought: Hi, welcome to Assistant! How can I assist you today?

Action: Search
Action Input: Who is Tomie?
Observation: ["Tomie Kawakami, better known mononymously as Tomie, is a character from the Japanese horror manga and film series of the same name created by Junji Ito. Tomie made her first appearance in Ito's 1987 manga Tomie, which was published in Monthly Halloween, a shōjo magazine.", 'Tomie Kawakami type: Film character.', 'Tomie Kawakami entity_type: fictional_universe, people.', 'Tomie Kawakami main_tab_text: Overview.', 'Tomie Kawakami kgmid: /g/11bvssns88.', 'Tomie Kawakami notable_aliases: Ayaka Nakamura, Mai Oyama, Marina, Miho Kaneshiro, Tomie, Tomie Iizuka, Tomie Kawakami.', 'Tomie Kawakami created_by: Junji Ito.', 'Tomie Kawakami first_appearance: Tomie (1987).', 'Tomie Kawakami last_appearance: Tomie - Takeover (2018).', 'Tomie Kawakami better known mononymously as Tomie, is a character from the Japanese horror manga and film s

{'input': 'hi i am tomie',
 'chat_history': '',
 'output': 'I hope that provides some insight into the Tomie series. Is there anything else you would like to know?'}

In [28]:
agent.invoke("what is my name?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Your name is Tomie.

> Finished chain.


{'input': 'what is my name?',
 'chat_history': 'Human: hi i am tomie\nAI: I hope that provides some insight into the Tomie series. Is there anything else you would like to know?',
 'output': 'Your name is Tomie.'}

In [29]:
agent.invoke("有什么好吃的泰国菜可以推荐给我吗?")



> Entering new AgentExecutor chain...
AI: 嗯，泰国菜有很多种，每种都有不同的特色。如果你喜欢辣的，我可以推荐你品尝一下冬阴功汤或者咖喱鸡，它们都是非常受欢迎的菜肴。如果你喜欢甜辣口味，那么芒果鸡或者青木瓜沙拉可能会更合你的口味。如果你想尝试一些有趣的菜肴，可以尝试一下泰式炸春卷或者泰式凉拌粉丝。总的来说，泰国菜具有丰富多样的口味，你可以根据自己的喜好选择。

> Finished chain.


{'input': '有什么好吃的泰国菜可以推荐给我吗?',
 'chat_history': 'Human: hi i am tomie\nAI: I hope that provides some insight into the Tomie series. Is there anything else you would like to know?\nHuman: what is my name?\nAI: Your name is Tomie.',
 'output': '嗯，泰国菜有很多种，每种都有不同的特色。如果你喜欢辣的，我可以推荐你品尝一下冬阴功汤或者咖喱鸡，它们都是非常受欢迎的菜肴。如果你喜欢甜辣口味，那么芒果鸡或者青木瓜沙拉可能会更合你的口味。如果你想尝试一些有趣的菜肴，可以尝试一下泰式炸春卷或者泰式凉拌粉丝。总的来说，泰国菜具有丰富多样的口味，你可以根据自己的喜好选择。'}

In [30]:
try:
    agent.invoke("这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

AI: 你的名字的最后一个字母是"e"。1998年的世界杯法国队夺冠了，他们在决赛中击败了巴西队。那是一个非常精彩的比赛，你可以看看相关的比赛录像来回味一下。 

> Finished chain.


In [31]:
agent.invoke("中国陕西西安现在的气温多少？截止目前我们聊了什么？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: What is the current temperature in Xi'an, China?
Observation: {'type': 'weather_result', 'temperature': '71', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '68%', 'wind': '17 mph', 'location': "Xi'An, Shaanxi, China", 'date': 'Friday 2:00 PM', 'weather': 'Mostly cloudy'}
Thought: Do I need to use a tool? Yes
Action: N/A
AI: As for our conversation, we discussed your name, Thai cuisine recommendations, and the 1998 World Cup winner.

> Finished chain.


{'input': '中国陕西西安现在的气温多少？截止目前我们聊了什么？',
 'chat_history': 'Human: hi i am tomie\nAI: I hope that provides some insight into the Tomie series. Is there anything else you would like to know?\nHuman: what is my name?\nAI: Your name is Tomie.\nHuman: 有什么好吃的泰国菜可以推荐给我吗?\nAI: 嗯，泰国菜有很多种，每种都有不同的特色。如果你喜欢辣的，我可以推荐你品尝一下冬阴功汤或者咖喱鸡，它们都是非常受欢迎的菜肴。如果你喜欢甜辣口味，那么芒果鸡或者青木瓜沙拉可能会更合你的口味。如果你想尝试一些有趣的菜肴，可以尝试一下泰式炸春卷或者泰式凉拌粉丝。总的来说，泰国菜具有丰富多样的口味，你可以根据自己的喜好选择。\nHuman: 这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？\nAI: 你的名字的最后一个字母是"e"。1998年的世界杯法国队夺冠了，他们在决赛中击败了巴西队。那是一个非常精彩的比赛，你可以看看相关的比赛录像来回味一下。',
 'output': 'As for our conversation, we discussed your name, Thai cuisine recommendations, and the 1998 World Cup winner.'}

- CHAT_CONVERSATIONAL_REACT_DESCRIPTION 使用了chatmodel
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION used chatmodel

In [32]:
from langchain_openai import  ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)
print(agent.agent.llm_chain.prompt.messages[3])

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['chat-conversational-react-description'] agent=ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Ass

In [33]:
agent.invoke("hi i am tony")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Tony, how can I assist you today?"
}
```

> Finished chain.


{'input': 'hi i am tony',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony, how can I assist you today?')],
 'output': 'Hello Tony, how can I assist you today?'}

In [34]:
agent.invoke("what is my name?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Your name is Tony."
}
```

> Finished chain.


{'input': 'what is my name?',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony, how can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Tony.')],
 'output': 'Your name is Tony.'}

In [35]:
agent.invoke("有什么好吃的泰国菜给我推荐一下？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "推荐泰国菜"
}
```
Observation: ['要说到泰菜里面Vivian最喜欢的一定要属菠萝炒饭啦！泰国香米的香气，搭配菠萝独有的酸酸甜甜的味道，真的是升华了整道炒饭的口感！而且这道菜营养均衡，有主食， ...', '这10道泰国菜美味又健康，你吃过几道？ · 青木瓜沙拉(ส้มตำ) · 泰式柚子沙拉(ยำส้มโอ) · 泰式春卷(ปอเปี๊ยะสด) · 清汤冬阴功(ต้มยำกุ้งน้ำใส) · 海南鸡饭(ข้าวมันไก่) · 柠檬 ...', '10道必尝的泰国菜 · Larb Moo · Phad Thai · Khao Soi · Kra Pao Moo Krob · Som Tam · Gai Pad Med Ma Muang · Hor Mok Pla ...', '1.冬阴功汤-经典口味，泰式热辣 · 2.青木瓜沙拉-世界十大最佳沙拉 · 3.泰式炒河粉-家喻户晓的菜肴 · 4.泰式猪脚饭-老字号，米其林推荐 · 5.打抛猪肉饭-民间美食 ...', '泰式椰汁鸡汤 · 寿喜烧 · 泰式烤鸡 · 泰式炒裸条 · 打抛叶 · 冬荫功汤 · 泰式青木瓜沙拉 · 海南鸡饭.', '香辣酸甜最够味，洛杉矶12家泰国菜推荐泰式料理可不只有冬阴功汤 · 1. Thai Bamboo Bistro | 家常便饭好去处 · 2. Mint Leaf Thai Cuisine | 吃完喝完去 ...', '（一）\u200b\u200b 作为泰国菜的代表——冬荫功汤，想必是很多朋友最为熟悉的泰国菜！ CNN推荐名单中，冬荫功汤稳居第一，一直颇受外国游客欢迎，冬荫功也是许多来泰国 ...', '纽约市泰餐餐厅 · 1. Bua Thai Ramen & Robata Grill. (847) · 2. Pure Thai Cookhouse. (1,548) · 3. Thai Villa. (229) · 4. Obao Midtown. (423) · 5. Up Thai · 6. VIV Thai ...', '排名前十的泰国菜——普吉岛最受欢迎的大

{'input': '有什么好吃的泰国菜给我推荐一下？',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony, how can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Tony.'),
  HumanMessage(content='有什么好吃的泰国菜给我推荐一下？'),
  AIMessage(content='一些值得推荐的泰国菜包括菠萝炒饭，这是一种香米炒饭，加入了菠萝，使得口味酸甜，很受欢迎。你也可以尝试冬阴功汤，这是一种著名的泰国辣汤，以及青木瓜沙拉，这是一种世界著名的沙拉，其中包括了辣椒，大蒜，豆类等食材。泰式炒粿条和绿咖喱鸡也是非常流行的选择。这些建议都来源于人们对泰国菜的喜爱和推荐，希望你会喜欢这些经典的泰国风味。')],
 'output': '一些值得推荐的泰国菜包括菠萝炒饭，这是一种香米炒饭，加入了菠萝，使得口味酸甜，很受欢迎。你也可以尝试冬阴功汤，这是一种著名的泰国辣汤，以及青木瓜沙拉，这是一种世界著名的沙拉，其中包括了辣椒，大蒜，豆类等食材。泰式炒粿条和绿咖喱鸡也是非常流行的选择。这些建议都来源于人们对泰国菜的喜爱和推荐，希望你会喜欢这些经典的泰国风味。'}

In [36]:
agent.invoke("我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "1998年世界杯冠军球队"
}
```
Observation: France national football team
Thought:```json
{
    "action": "Final Answer",
    "action_input": "你名字的最后一个字母是Y。1998年的世界杯冠军是法国国家足球队。"
}
```

> Finished chain.


{'input': '我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？',
 'chat_history': [HumanMessage(content='hi i am tony'),
  AIMessage(content='Hello Tony, how can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Tony.'),
  HumanMessage(content='有什么好吃的泰国菜给我推荐一下？'),
  AIMessage(content='一些值得推荐的泰国菜包括菠萝炒饭，这是一种香米炒饭，加入了菠萝，使得口味酸甜，很受欢迎。你也可以尝试冬阴功汤，这是一种著名的泰国辣汤，以及青木瓜沙拉，这是一种世界著名的沙拉，其中包括了辣椒，大蒜，豆类等食材。泰式炒粿条和绿咖喱鸡也是非常流行的选择。这些建议都来源于人们对泰国菜的喜爱和推荐，希望你会喜欢这些经典的泰国风味。'),
  HumanMessage(content='我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？'),
  AIMessage(content='你名字的最后一个字母是Y。1998年的世界杯冠军是法国国家足球队。')],
 'output': '你名字的最后一个字母是Y。1998年的世界杯冠军是法国国家足球队。'}

- OPENAI_FUNCTIONS，使用openai的函数调用来实现的，只支持openai模型。
- OPENAI_FUNCTIONS, implemented using openai's function calls, only supports openai models.

In [37]:
from langchain_openai import  ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.prompt.messages)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['openai-functions'] agent=OpenAIFunctionsAgent(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000187EF406480>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000187EF4066C0>, model_name='gpt-4-1106-preview', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://ai-yyds.com/v1', openai_proxy='https://ai-yyds.com/v1'), tools=[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='e8c972251bd371ef03c23b9bc9e1ad49cb8cb7823befbfda0c8980d7a66b5fa0', aiosession=None)>, coroutine=<bound method 

In [38]:
try:
    agent.invoke("今天比特币的价格是多少？它的价格开平方是多少？")
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `Search` with `
Bitcoin price today`


63,788.00Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null. (request id: 20240920141958695731667QDbRHg8U) (request id: 2024092006195854962508799305644)", 'type': 'invalid_request_error', 'param': 'messages.[2].content', 'code': None}}


- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 对输出做了结构化处理
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION Structures the output.

In [39]:
from langchain_openai import  ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
    tool
)
from langchain.memory import ConversationBufferMemory

#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)

@tool
def search(query: str) -> str:
    """只有需要了解实时信息或不知道的事情的时候才会使用这个工具."""
    serp = SerpAPIWrapper()
    return serp.run(query)
    
agent = initialize_agent(
    [search],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['structured-chat-zero-shot-react-description'] agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nsearch: search(query: str) -> str - 只有需要了解实时信息或不知道的事情的时候才会使用这个工具., args: {{\'query\': {{\'title\': \'Query\', \'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or search\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nActio

In [40]:
agent.invoke({"input":"what is langchain?"})



> Entering new AgentExecutor chain...
Thought: Langchain is not a widely-known term or entity that would fall into common knowledge as of my last update, and it could refer to a specific product, project, or concept that has arisen after my last update. To provide accurate information, I would need to conduct a search.

Action:
```json
{
  "action": "search",
  "action_input": {
    "query": "What is langchain?"
  }
}
```


Observation: ["LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.", 'LangChain entity_type: kp3_verticals.', 'LangChain kgmid: /g/11kjpl7_60.', 'LangChain initial_release_date: October 2022.', 'LangChain developer_s: Harrison Chase.', 'LangChain license: MIT License.', 'LangChain repository: github.com/langchain-ai/l

{'input': 'what is langchain?',
 'chat_history': [HumanMessage(content='what is langchain?'),
  AIMessage(content='Langchain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It was initially released in October 2022 and is developed by Harrison Chase. Langchain is written in Python and JavaScript, and its stable release as of April 2024 is version 0.1.16. The framework provides tools for developers to connect language models with external data sources and is supported by an active community. It is particularly focused on chaining interoperable components and building controllable agentic workflows through a component called LangGraph. The framework is available on GitHub and is licensed under the MIT License.')],
 'output': 'Langchain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It was initially released in October 2022 and is developed by Harrison Chase

### 增加的具体几种agent
### Several different types of agent
- https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
<hr>